In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k, ndcg_at_k, ap_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

# Функция генерации фичей
from src.features import generate_fs

# Функция быстрого старта
from src.recommendation import tlvl_recommender

In [9]:
data = pd.read_csv('../raw_data/retail_train.csv')
item_features = pd.read_csv('../raw_data/product.csv')
user_features = pd.read_csv('../raw_data/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_train_lvl_2 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [5]:
recommender, pred = tlvl_recommender(data, week_sep=3, item_features=item_features, 
                                     user_features=user_features, N=100)

C:\Users\Diana\Documents\Recsys\Final Project\src\utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cost'] = data['sales_value'] / data['quantity']
C:\Users\Diana\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

C:\Users\Diana\Documents\Recsys\Final Project\src\features.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values(['week_no', 'day', 'trans_time'], inplace=True)
C:\Users\Diana\Documents\Recsys\Final Project\src\features.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['value'] = (data['sales_value'] - data['retail_disc']) / data['quantity']
C:\Users\Diana\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sample

In [20]:
def get_recommendation(user, N=5):
    if user in data_train_lvl_1['user_id'].unique() and \
            user in data_train_lvl_2['user_id'].unique() and \
            user in data_val_lvl_2['user_id'].unique():
        return pred.loc[pred['user_id'] == user, 'pred'].values[0]
    elif (user in data_train_lvl_1['user_id'].unique() and \
            user in data_train_lvl_2['user_id'].unique()) or \
        (user in data_train_lvl_1['user_id'].unique() and \
            user in data_val_lvl_2['user_id'].unique()):
        return recommender.get_own_recommendations(user, N=N)
    else:
        return recommender.extend_with_top_popular([], N=N)

In [22]:
get_recommendation(1)

array([  856942,  5577022,  1062002,   865456,   914190,  8090541,
         883616,   958046,  1004906,   990656,   971585,   979707,
         940947,  1005186,  1115576,  1041796,   978974,   933913,
        1050310,   986947,  1094955,  9655212, 10150194,  1022053,
        1020682,   940631,  1075074,   911454,  1105433,  1025641,
        1124029,  1082269,   965766,  1056509,   847982,  1035805,
        1082212,  6034857,  1006184,  5569374,   872137,   986912,
         844818,   865178,  1021164,   934369, 10149640,  1033142,
        9527290,  1137775,   904360,  1013167,  5568729,  1100972,
        1053282,   997796,   992986,   977658,  1022254,  1005274,
         931136,   979792,   945805,  8293343,  1011736,  1135096,
         888178,   830775,  1089023,  1055201, 12384953,  1096120,
       12385916,   924423, 12385374,   869857,   845078,   878285,
         859191,   848761, 15926844,   991024,   953561,   876042,
        1043064,  5582712,   857006,   834631,  8090513,   835